# Difference in Differences with `pymc` models

<div class="alert alert-warning">

Warning

This example is in-progress! We are still refining the difference in differences analysis code, so do not trust the results yet. Expect improvements soon.

</div>

In [1]:
import arviz as az

import causalpy as cp

In [2]:
%load_ext autoreload
%autoreload 2
# %config InlineBackend.figure_format = 'svg'

## Load data

In [3]:
df = cp.load_data("did")
df.head()

,group,t,unit,post_treatment,y
0,0,0.0,0,False,0.897122
1,0,1.0,0,True,1.961214
2,1,0.0,1,False,1.233525
3,1,1.0,1,True,2.752794
4,0,0.0,2,False,1.149207


## Run the analysis

In [4]:
result = cp.pymc_experiments.DifferenceInDifferences(
    df,
    formula="y ~ 1 + group + t + group:post_treatment",
    time_variable_name="t",
    group_variable_name="group",
    treated=1,
    untreated=0,
    prediction_model=cp.pymc_models.LinearRegression(),
)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2 seconds.
Sampling: [beta, sigma, y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]
Sampling: [y_hat]


In [5]:
fig, ax = result.plot()

In [6]:
result.summary()

===========================Difference in Differences============================
Formula: y ~ 1 + group + t + group:post_treatment

Results:
Causal impact = 0.50, $CI_{94\%}$[0.41, 0.61]
Model coefficients:
Intercept                     1.08, 94% HDI [1.03, 1.13]
group                         0.16, 94% HDI [0.09, 0.23]
group:post_treatment[T.True]  0.50, 94% HDI [0.41, 0.61]
t                             0.99, 94% HDI [0.92, 1.06]
sigma                         0.08, 94% HDI [0.07, 0.10]


In [7]:
ax = az.plot_posterior(result.causal_impact, ref_val=0)
ax.set(title="Posterior estimate of causal impact");